In [2]:
import pandas as pd
import json
import pprint
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
import os
import csv
import datetime
import numpy as np

In [3]:
us_csv_full = "USA_AllYears.csv"
us_csv_glance = "USA_At-a-Glance.csv"
us_full = pd.read_csv(us_csv_full)
us_glance = pd.read_csv(us_csv_glance)

In [4]:
us_full.head()

,Reporter,Partner,Product categories,Indicator Type,Indicator,2018,2017,2016,2015,2014,...,1997,1996,1995,1994,1993,1992,1991,1990,1989,1988
0,United States,...,...,Development,GDP (current US$ Mil),20544343.46,19485393.85,18707188.23,18219297.58,17521746.53,...,8577554.46,8073122.0,7639749.0,7287236.0,6858559.0,6520327.0,6158129.0,5963144.0,5641580.0,5236438.0
1,United States,...,...,Export,No. Of Export partners,224.00,224.00,225.00,224.00,225.00,...,210.00,211.0,216.0,215.0,214.0,211.0,192.0,NaN,NaN,NaN
2,United States,...,...,Export,No. Of Export products,4533.00,4523.00,4563.00,4559.00,4560.00,...,4916.00,4918.0,4986.0,4987.0,4981.0,4979.0,4984.0,NaN,NaN,NaN
3,United States,...,...,Import,No. Of Import partners,223.00,222.00,222.00,222.00,221.00,...,205.00,207.0,209.0,206.0,210.0,208.0,188.0,NaN,NaN,NaN
4,United States,...,...,Import,No. Of Import products,4531.00,4527.00,4558.00,4561.00,4555.00,...,4931.00,4934.0,5000.0,4991.0,4994.0,4993.0,4997.0,NaN,NaN,NaN


In [5]:
us_glance.head()

,Reporter,Year,Partner,Product categories,Indicator Type,Indicator,Indicator Value,Unnamed: 7
0,United States,2018.0,Canada,All Products,Export,Trade (US$ Mil)-Top 5 Export Partner,299744.49250,NaN
1,United States,2018.0,Mexico,All Products,Export,Trade (US$ Mil)-Top 5 Export Partner,265434.78250,NaN
2,United States,2018.0,China,All Products,Export,Trade (US$ Mil)-Top 5 Export Partner,120147.86570,NaN
3,United States,2018.0,Japan,All Products,Export,Trade (US$ Mil)-Top 5 Export Partner,75226.08562,NaN
4,United States,2018.0,United Kingdom,All Products,Export,Trade (US$ Mil)-Top 5 Export Partner,66293.66445,NaN


In [6]:
# Grab data for export pie chart
export_categories = us_full.loc[(us_full["Indicator Type"] == "Export") & (us_full["Indicator"] == "Export(US$ Mil)")]
export_cats = export_categories.loc[(export_categories["Product categories"] != "Capital goods") & (export_categories["Product categories"] != "Consumer goods")
                            & (export_categories["Product categories"] != "Intermediate goods") & (export_categories["Product categories"] != "Raw materials")]
export_cats = export_cats.drop(columns=['1988', '1989', '1990'])
#export_categories.to_csv('data/tradeData.csv')
export_cats = export_cats[['Product categories', '2018', '2008']]
export_cats = export_cats.rename(columns={'Product categories': 'categories', '2018': 'year_2018', '2008': 'year_2008'})
export_cats
#export_cats.to_csv('data/tradeData.csv')


,categories,year_2018,year_2008
21,Animal,29260.67,20737.39
23,Chemicals,170946.35,141335.02
25,Food Products,46788.09,28426.28
26,Footwear,2089.32,1358.90
27,Fuels,192681.24,76711.36
28,Hides and Skins,4106.00,4276.98
30,Mach and Elec,376388.94,365173.93
31,Metals,73519.91,74834.35
32,Minerals,10387.62,8708.37
33,Miscellaneous,189960.05,139000.18


In [9]:
# Grab data for export pie chart
export_percentages = us_full.loc[(us_full["Indicator Type"] == "Export") & (us_full["Indicator"] == "Export Product share(%)")]
export_percentages = export_percentages.drop(columns=['1988', '1989', '1990'])
#export_percentages

In [10]:
# Grab data for export pie chart
export_perc_products = export_percentages.loc[(export_percentages["Product categories"] != "Capital goods") & (export_percentages["Product categories"] != "Consumer goods")
                            & (export_percentages["Product categories"] != "Intermediate goods") & (export_percentages["Product categories"] != "Raw materials")]

#export_perc_products

In [9]:
# Grab data for export pie chart
export_perc_categories = export_percentages.loc[(export_percentages["Product categories"] == "Capital goods") | (export_percentages["Product categories"] == "Consumer goods")
                            | (export_percentages["Product categories"] == "Intermediate goods") | (export_percentages["Product categories"] == "Raw materials")]
export_perc_categories

,Reporter,Partner,Product categories,Indicator Type,Indicator,2018,2017,2016,2015,2014,...,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991
69,United States,World,Capital goods,Export,Export Product share(%),32.44,33.55,34.36,34.47,33.10,...,51.60,51.13,50.36,49.04,46.77,45.75,46.38,45.48,45.08,44.66
71,United States,World,Consumer goods,Export,Export Product share(%),25.91,25.92,25.53,25.86,27.31,...,19.88,19.42,19.29,19.18,19.41,19.20,20.31,20.04,19.70,18.53
76,United States,World,Intermediate goods,Export,Export Product share(%),19.46,19.53,19.59,19.87,19.89,...,18.54,19.02,19.48,19.72,19.76,20.80,19.80,20.34,20.04,21.17
82,United States,World,Raw materials,Export,Export Product share(%),11.41,10.23,9.42,9.09,10.06,...,6.54,6.67,7.20,8.29,10.11,10.48,9.48,9.87,10.82,11.13


In [10]:
# Grab data for export pie chart
export_perc_categories = export_categories.loc[(export_categories["Product categories"] == "Capital goods") | (export_categories["Product categories"] == "Consumer goods")
                            | (export_categories["Product categories"] == "Intermediate goods") | (export_categories["Product categories"] == "Raw materials")]
export_perc_categories

,Reporter,Partner,Product categories,Indicator Type,Indicator,2018,2017,2016,2015,2014,...,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991
22,United States,World,Capital goods,Export,Export(US$ Mil),540285.30,518558.06,498512.90,517758.98,536096.70,...,403450.75,354209.03,342680.23,337165.21,291286.89,266727.88,237597.64,211362.95,201668.22,188282.73
24,United States,World,Consumer goods,Export,Export(US$ Mil),431527.79,400719.11,370436.52,388367.79,442287.78,...,155457.15,134506.31,131254.69,131840.83,120873.46,111911.89,104059.18,93154.36,88110.70,78103.68
29,United States,World,Intermediate goods,Export,Export(US$ Mil),324103.85,301869.80,284221.32,298377.59,322180.79,...,144981.66,131745.92,132573.68,135606.02,123059.34,121234.92,101424.40,94517.84,89627.01,89234.65
35,United States,World,Raw materials,Export,Export(US$ Mil),189985.11,158069.68,136670.47,136449.47,162869.32,...,51155.46,46189.31,49002.63,56990.81,62947.48,61121.13,48544.94,45862.88,48402.54,46927.57


In [11]:
Export_product_share_2018 = export_percent[['Reporter', 'Partner', 'Product categories', 'Indicator Type', 'Indicator', '2017']]
Export_product_share_2018['2017'].sum()

NameError: name 'export_percent' is not defined

In [12]:
# Import data for pie chart
import_categories = us_full.loc[(us_full["Indicator Type"] == "Import") & (us_full["Indicator"] == "Import(US$ Mil)")]
import_categories = import_categories.drop(columns=['1988', '1989', '1990'])
import_categories

,Reporter,Partner,Product categories,Indicator Type,Indicator,2018,2017,2016,2015,2014,...,2000,1999,1998,1997,1996,1995,1994,1993,1992,1991
42,United States,World,Capital goods,Import,Import(US$ Mil),899657.83,835944.76,775806.65,794043.56,771189.05,...,426299.50,371274.93,328812.88,311271.24,284988.62,277256.18,237334.26,199992.25,176566.16,157381.24
43,United States,World,Consumer goods,Import,Import(US$ Mil),948294.30,885215.06,848244.16,862734.52,850923.70,...,438558.60,377702.88,337962.54,306688.79,274954.13,258955.12,240854.15,213823.17,195238.70,177336.99
44,United States,World,Intermediate goods,Import,Import(US$ Mil),400762.12,356603.46,347230.18,355394.35,373263.57,...,180760.31,162081.79,154306.97,143443.15,130905.85,124826.22,109666.72,93338.82,86577.21,81591.41
45,United States,World,Raw materials,Import,Import(US$ Mil),254046.37,227562.20,189452.96,216179.07,341332.38,...,123872.23,88513.57,75508.56,92024.39,85317.61,76164.78,68701.93,65952.16,65463.03,63688.55
